<a href="https://colab.research.google.com/github/ChelseaOwiti/Colposcopy-Scan/blob/main/Copy_of_ISP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install -q keras
!pip install dicom

%matplotlib inline

import numpy as np
import pandas as pd 
import keras

import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt
 
from skimage import measure, morphology 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

train_path = '/content/drive/MyDrive/train_new/train'
patients = os.listdir(train_path)
patients.sort()




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#load images in folder path
def load_scan(path):
  slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
  slices.sort(key = lambda x:float(x.ImagePositionPatient[2]))
  try:
    slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slice[1].ImagePositionPatient[2])
  except:
    slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
  
  for s in slices:
    s.SliceThickness = slice_thickness

  return slices

In [ ]:
def get_pixels_hu(slices):
  image = np.stack([s.pixel_array for s in slices])

  # convert to int 16
  image = image.astype(np.int16)

  # set outside of scan pixels to 0
  #intercept is usually -1024, so air is approx 0

  image[image == -2000] = 0
   
  #convert to Hounsfield units (hu)

  for slice_number in range(len(slices)):
    intercept = slices[slice_number].RescaleIntercept

    slope = slices[slice_number].RescaleIntercept
    if slope !=1:
      image[slice_number] = slope * image[slice_number].astype(np.float64)
      image[slice_number] =  image[slice_number].astype(np.int16)
      image[slice_number] += np.int16(intercept)
  return np.array(image, dtype =np.int16)

In [ ]:
#test patient 1
first_patient = load_scan(patients[0])
first_patient_pixels = get_pixels_hu(first_patient)
plt.hist(first_patient_pixels.flatten(), bins=80, color='c')

plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

#show som slice in midle
plt.imshow(first_patient_pixels[80], cmap=plt.cm.gray)
plt.show()

FileNotFoundError: ignored